In [5]:
import openai
from getpass import getpass
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import datetime
import csv
openai.api_key = open('apikey.txt', 'r').read()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [46]:
finalversionx3 = pd.read_csv("finalversionx3.csv")
finalversionx3_copy = finalversionx3.copy()



In [7]:
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo", 
    messages =[{"role": "user", "content": "1+1"}]
)

print(completion)

{
  "id": "chatcmpl-7z7WnquerkjLVJovdh3KWYhC1u6IQ",
  "object": "chat.completion",
  "created": 1694801301,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "1+1 equals 2."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 10,
    "completion_tokens": 7,
    "total_tokens": 17
  }
}


In [8]:
reply_content = completion.choices[0].message.content

In [54]:
# Define a function to truncate the conversation to fit within the token limit
def truncate_conversation(conversation, max_tokens=3000):
    total_tokens = 0
    truncated_conversation = []

    for message in conversation:
        message_tokens = len(message["content"].split())  # Approximate tokens, adjust as needed
        total_tokens += message_tokens

        if total_tokens <= max_tokens:
            truncated_conversation.append(message)
        else:
            break

    return truncated_conversation

In [63]:
# Create an empty list to store the summaries
summaries = []

# Create a new column 'school_summary' in your DataFrame
finalversionx3_copy['school_summary'] = ""

# Iterate through each unique school in your DataFrame
for school_name in finalversionx3_copy['school_name'].unique():
    # Filter comments for the current school
    school_comments = finalversionx3_copy[(finalversionx3_copy['school_name'] == school_name)]

    # Create a conversation for the current school's comments
    conversation = [{"role": "system", "content": f"Generate a summary in bullet points for these comments (prefer to consider comments with strong arguments) about {school_name}:"}]

    for comment in school_comments['comment']:
        if pd.notna(comment):
            conversation.append({"role": "user", "content": comment})
    for comment in school_comments['commentgg']:
        if pd.notna(comment):
            conversation.append({"role": "user", "content": comment})

     # Truncate the conversation
    truncated_conversation = truncate_conversation(conversation)

    # Convert the truncated conversation to a format suitable for OpenAI API
    messages = [{"role": message["role"], "content": message["content"]} for message in truncated_conversation]

    # Use ChatGPT to generate a summary for the conversation
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    
    # Extract the summary from the response
    summary = completion.choices[0].message.content

    # Append the summary to the 'summaries' list
    summaries.append(summary)


APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
)

In [64]:
# Add the summaries to the DataFrame by matching school names
finalversionx3_copy['school_summary'] = finalversionx3_copy['school_name'].map(
    {name: summary for name, summary in zip(finalversionx3_copy['school_name'].unique(), summaries)}
)

In [65]:
finalversionx3_copy.to_csv("finalversionx4x2.csv", index = False, quoting=csv.QUOTE_ALL)